<a href="https://colab.research.google.com/github/Cinex10/faq_extractor/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install sentence-transformers

In [2]:
# !pip -q install -U jax jaxlib

In [3]:
!pip install -qU scikit-learn

## Importing libraries

In [39]:
import torch
from sentence_transformers import SentenceTransformer
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN,HDBSCAN
from bokeh.transform import linear_cmap
from bokeh.palettes import Spectral11,Sunset3

# NLP Phase

## Corpus

In [5]:
dataset = np.array([
"What's the cost of a standard room?",
"Can you provide information on your room types?",
"What are your hotel's check-in and check-out timings?",
"Are there any extended stay discounts available?",
"Does the room rate include complimentary breakfast?",
"Is it permissible to have pets in the hotel?",
"Is there a fitness facility or gym within the hotel premises?",
"How do I request a late check-out?",
"Is there an airport shuttle service available?",
"Do you offer rooms with separate smoking and non-smoking options?",
"May I reserve a room with a scenic view?",
"Could you detail the room service options and operational hours?",
"Are there recommended nearby restaurants and attractions?",
"Does the room rate include Wi-Fi access, and what's the network name?",
"Could you explain the hotel's reservation cancellation policy?",
"Do you offer any special packages for events or celebrations?",
"Can I request an additional bed or crib for my room?",
"What's the distance from the hotel to the closest train station?",
"Are there rooms designed for guests with mobility challenges?",
"Is there a pool or spa on the hotel premises?",
"How can I arrange transportation to local attractions?",
"What's the policy for holding luggage before check-in or after check-out?",
"Can I choose my room preferences when I book?",
"Do you offer discounts for senior citizens or AARP members?",
"Is there a concierge desk to assist with local recommendations?",
"What's the process for addressing lost or left-behind items?",
"Are there any in-room amenities like a mini-fridge or microwave?",
"What's your policy on early check-in requests?",
"Can I get assistance with tour bookings or tickets?",
"How often are the rooms cleaned and serviced?",
"Is there a dress code for the hotel's dining areas?",
"Are there any on-site meeting or event facilities?",
"Can I book a room with a balcony or terrace?",
"Is there a quiet time policy in the hotel?",
"Can you provide recommendations for nearby shopping areas?",
"Do you offer laundry or dry-cleaning services?",
"Is room keycard access required for specific hotel areas?",
"Are there designated smoking areas on the property?",
"Can I arrange for a wake-up call in the morning?",
"What's the hotel's policy on lost and found items?",
"Do you provide discounts for military personnel or veterans?",
"Are there any outdoor seating or dining options on-site?",
"Can I request a specific floor or room location?",
"How can I contact the front desk for assistance?",
"Do you offer transportation services to the city center?",
"What's the process for extending my stay?",
"Can I pre-order amenities like flowers or champagne for special occasions?",
"Are there any security measures in place within the hotel?",
"Can I leave my luggage at the front desk before check-in?",
"Do you offer babysitting or childcare services?",
"Is there a business center with printing and copying facilities?",
"How is the hotel handling COVID-19 safety measures?",
"What's the policy on lost or misplaced room keys?",
"Are there any hidden fees not included in the room rate?",
"How can I book a room for a large group or event?",
"Do you provide airport pick-up services?",
"Can I access the hotel's fitness center 24/7?",
"What's the procedure for early check-out?",
"Is there a curfew or quiet hours in the hotel?",
"Are there in-room dining options available?",
"Do you provide transportation to local hospitals or medical facilities?",
"Can I store valuable items in a hotel safe?",
"Are there any nearby scenic walking or jogging trails?",
"What's the policy for receiving mail or packages?",
"Is there a loyalty program for frequent guests?",
"Do you have a lost and found department for guests?",
"Are there any in-room entertainment options?",
"Can I change my room type upon arrival if available?",
"Are there designated pet-friendly rooms?",
"How can I access the hotel's guest reviews?",
"Is there a courtesy shuttle to popular tourist attractions?",
"Can I arrange for early morning coffee or tea delivery?",
"Do you offer assistance for planning proposals or special occasions?",
"Are there any electric vehicle charging stations on-site?",
"Can I have flowers or gifts delivered to my room?",
"What's your policy on accommodating dietary restrictions for meals?",
"Can I use a hotel phone for local calls?",
"Are there discounts for government employees or diplomats?",
"Is there a library or reading room within the hotel?",
"How can I access the hotel's event calendar?",
"Can I upgrade my room after checking in?",
"Do you provide special amenities for honeymooning couples?",
"Are there any cultural or artistic events in the area?",
"How do I access the hotel's spa or wellness center?",
"Are there any nearby scenic walking or jogging trails?",
"Can I book a room for an extended period, such as a month?",
"Is there a designated area for picnics or outdoor dining?",
"How do I access room service menus and pricing?",
"Are there options for in-room cooking or kitchenettes?",
"Can I request a late-night check-in or early check-out?",
"Is there a security escort service for late-night arrivals?",
"How can I receive notifications about hotel promotions?",
"Can I participate in a hotel loyalty program upon check-in?",
"Are there options for in-room massages or spa treatments?",
"Can I get assistance with planning local sightseeing tours?",
"How can I arrange a private dining experience within the hotel?",
"Are there designated meeting or event coordinators on staff?",
"Is there a quiet area for reading or studying within the hotel?",
"Can I schedule wake-up calls with specific music or sounds?",
"Are there options for in-room cooking or kitchenettes?",
"How can I access the hotel's full menu of services and amenities?",
"Can I book a conference room or business meeting space?",
"Is there a car rental service available at the hotel?",
"Can I have special dietary preferences noted for dining?",
"How can I access the hotel's digital concierge services?",
"Are there options for arranging surprise gifts or decorations?",
"Is there a helipad or space for helicopter arrivals?",
"Can I request a designated smoking balcony or terrace?",
"How can I access the hotel's art or sculpture gallery?",
"Are there options for arranging live music or entertainment?",
"Can I receive information on hotel sustainability efforts?",
"Is there a designated area for meditation or yoga?",
"How can I access the hotel's library of books or movies?",
"Can I schedule room cleanings at specific times?",
"Are there options for in-room cooking lessons or classes?",
"How can I access the hotel's cultural or historical exhibits?",
"Can I have a private event with personal catering?",
"Is there a photography service for special occasions?",
"How can I access the hotel's VIP or concierge lounge?",
"Are there options for in-room babysitting or childcare?",
"Can I receive detailed maps of local walking tours?",
"Is there a shuttle to nearby golf courses?",
"How can I access information on hotel awards and accolades?",
"Can I have a personal trainer for in-room workouts?",
"Is there a designated area for art or craft workshops?",
"How can I access information on hotel partnerships or affiliations?",
"Can I schedule in-room laundry or dry-cleaning services?",
"Are there options for in-room language or cooking lessons?",
"Can I access the hotel's list of recommended local artists?",
"How can I receive a copy of the hotel's sustainability report?",
])

## Text Preprocessing

In [6]:
from sklearn.decomposition import PCA

def reduce_dim(x,n_components=5):
  pca = PCA(n_components=n_components)
  reduced_data = pca.fit_transform(x)
  return torch.from_numpy(reduced_data)

In [7]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

## Calculating sentences embedings

In [8]:
emd = sbert_model.encode(dataset)

In [9]:
embeding_matrix = torch.tensor(emd)

## Similary matrix

In [10]:
similarity_matrix = torch.nn.functional.cosine_similarity(embeding_matrix.unsqueeze(1), embeding_matrix.unsqueeze(0),dim=2)

In [11]:
# Dimensionality reducing into 2 components
tsne = TSNE(n_components=2, perplexity=30, random_state=0)
reduced_vectors = tsne.fit_transform(similarity_matrix)

## Visualize similarities

In [12]:
colors = plt.cm.rainbow(np.linspace(0, 1, dataset.shape[0]))

source = ColumnDataSource(data={
    'x': reduced_vectors[:, 0],
    'y': reduced_vectors[:, 1],
    'labels': dataset,
    'colors': colors,
})

# Create a Bokeh figure
p = figure(width=1800, height=600, title="Question Similarity Visualization")

# Add a scatter plot with tooltips
scatter = p.circle('x', 'y', size=15, source=source, fill_color="colors", line_color="colors")
# Define the tooltip style with HTML/CSS attributes
tooltips = """
Question:
<div style="font-size: 16px; color: darked;font-weight: bold;">
@labels
</div>
"""

# Add a hover tool with the defined tooltips
hover = HoverTool(renderers=[scatter], tooltips=tooltips)
p.add_tools(hover)

# Enable notebook display mode
output_notebook()

# Show the plot
show(p)

# Clustering Phase

## DBSCAN

In [13]:
eps = 0.12
min_samples = 2

In [14]:
dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric="precomputed")  # "precomputed" for using a precomputed similarity matrix

In [15]:
dissimilarity_matrix = 1 - similarity_matrix
dissimilarity_matrix[dissimilarity_matrix < 0] = 0
labels = dbscan.fit_predict(dissimilarity_matrix)  # Convert similarity to dissimilarity (1 - similarity) for DBSCAN

In [16]:
labels

array([-1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,  2,
       -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1,  3, -1,
       -1, -1, -1, -1, -1, -1,  1,  2, -1, -1, -1,  4, -1, -1, -1, -1, -1,
       -1,  5, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  4,
       -1, -1, -1,  6, -1, -1,  5, -1, -1, -1, -1, -1, -1, -1,  6, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  6, -1, -1, -1,  5,
        3, -1, -1,  5, -1, -1,  5, -1,  6, -1, -1])

In [55]:
def get_clusters_content(labels):
  clusters = {}

  for index,label in enumerate(labels):
    if label == -1:
      continue
    question = dataset[index]
    cluster_key = f'cluster_{label}'
    data = clusters.get(cluster_key)
    # print(f'index : {index} ,label : {label},data : {data}')
    if data is None:
      entry = {
          cluster_key : [question]
      }
    else:
      data.append(question)
      entry = {
          cluster_key : data
      }
    clusters.update(entry)

  return clusters

In [19]:
# Create a Bokeh figure
p = figure(width=1800, height=600, title="Cluster Visualization")

# Define a colormap for clusters
mapper = linear_cmap(field_name='cluster_label', palette=Spectral11, low=min(labels), high=max(labels))

# Create a data source for Bokeh
source = ColumnDataSource(data={
    'x': reduced_vectors[:, 0],
    'y': reduced_vectors[:, 1],
    'cluster_label': labels,  # Replace with your cluster labels
    'questions' : dataset,

})

# Create scatter plot glyphs
scatter = p.scatter(x='x', y='y', size=12, source=source, line_color=mapper, fill_color=mapper)
# Define the tooltip style with HTML/CSS attributes
tooltips = """
Question:
<div style="font-size: 16px; color: darked;font-weight: bold;">
@questions
</div>
"""

# Add a hover tool with the defined tooltips
hover = HoverTool(renderers=[scatter], tooltips=tooltips)
p.add_tools(hover)

# Enable notebook display mode
# Customize the plot as needed (e.g., axis labels, legends, tooltips)
p.xaxis.axis_label = "X-axis"
p.yaxis.axis_label = "Y-axis"

output_notebook()
# Show the plot
show(p)


## HDBSCAN

In [20]:
hdbscan = HDBSCAN(min_samples=2,min_cluster_size=2,metric="precomputed")

In [60]:
dissimilarity_matrix = 1 - similarity_matrix
dissimilarity_matrix[dissimilarity_matrix < 0] = 0
labels = hdbscan.fit_predict(dissimilarity_matrix)

In [61]:
labels

array([-1, 28, 29,  6, -1, -1, 19,  3,  1, -1, 13, 31, 20, -1, 18, 15, -1,
       -1, -1, 19, 10, 12, 24,  2, -1, -1, -1,  4, 10, -1, -1, 17, 13, 16,
       11, 22, 28,  8,  5, 18,  2, 20, 28, -1, 11,  6, 15, 28, 12,  7, -1,
       -1, 18, -1, -1,  1, -1,  4, 16, -1, -1, -1,  0, -1, -1, 18, -1, 24,
        7, 29, 10,  5, 15, -1, -1, 23, -1,  2, 16, 29, 30, -1, 14, 19,  0,
       -1, 20, 31, 26,  3,  3, 29, 29, 19, 10, -1, 17, 16,  9, 26, -1, -1,
       -1, 23, -1, 15,  1,  8, 25,  9, 21, -1, 16, 30, 27, 25, -1, 15, 29,
        7, 10, -1, 29, -1, 14, 29, 22, 27, -1, 21])

In [41]:
# Create a Bokeh figure
p = figure(width=1800, height=600, title="Cluster Visualization")

# Define a colormap for clusters
mapper = linear_cmap(field_name='cluster_label', palette=Spectral11, low=min(labels), high=max(labels))

# Create a data source for Bokeh
source = ColumnDataSource(data={
    'x': reduced_vectors[:, 0],
    'y': reduced_vectors[:, 1],
    'cluster_label': labels,  # Replace with your cluster labels
    'questions' : dataset,

})

# Create scatter plot glyphs
scatter = p.scatter(x='x', y='y', size=12, source=source, line_color=mapper, fill_color=mapper)
# Define the tooltip style with HTML/CSS attributes
tooltips = """
Question:
<div style="font-size: 16px; color: darked;font-weight: bold;">
@questions
</div>
"""

# Add a hover tool with the defined tooltips
hover = HoverTool(renderers=[scatter], tooltips=tooltips)
p.add_tools(hover)

# Enable notebook display mode
# Customize the plot as needed (e.g., axis labels, legends, tooltips)
p.xaxis.axis_label = "X-axis"
p.yaxis.axis_label = "Y-axis"

output_notebook()
# Show the plot
show(p)


In [62]:
clusters = get_clusters_content(labels)

In [67]:
for name,questions in clusters.items():
  print(name)
  for qst in questions:
    print('\t - ',qst)
  print('\n')

cluster_28
	 -  Can you provide information on your room types?
	 -  Is room keycard access required for specific hotel areas?
	 -  Can I request a specific floor or room location?
	 -  Are there any security measures in place within the hotel?


cluster_29
	 -  What are your hotel's check-in and check-out timings?
	 -  How can I access the hotel's guest reviews?
	 -  How can I access the hotel's event calendar?
	 -  How can I receive notifications about hotel promotions?
	 -  Can I participate in a hotel loyalty program upon check-in?
	 -  How can I access the hotel's VIP or concierge lounge?
	 -  How can I access information on hotel awards and accolades?
	 -  How can I access information on hotel partnerships or affiliations?


cluster_6
	 -  Are there any extended stay discounts available?
	 -  What's the process for extending my stay?


cluster_19
	 -  Is there a fitness facility or gym within the hotel premises?
	 -  Is there a pool or spa on the hotel premises?
	 -  How do I acc

# Exporting Bokeh Plots